## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
os.chdir("C:\\Users\\Reese\\DataClass\\6-World_Weather_Analysis")

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Indian Head,US,38.6001,-77.1622,50.16,59,100,5.75,overcast clouds
1,1,Albany,US,42.6001,-73.9662,36.63,82,87,2.93,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,78.75,70,13,6.62,few clouds
3,3,Saint-Philippe,RE,-21.3585,55.7679,76.59,84,37,8.05,scattered clouds
4,4,Cidreira,BR,-30.1811,-50.2056,74.50,87,99,2.71,overcast clouds


In [9]:
os.chdir("C:\\Users\\Reese\\DataClass\\6-World_Weather_Analysis\\Vacation_Search")

In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,78.75,70,13,6.62,few clouds
3,3,Saint-Philippe,RE,-21.3585,55.7679,76.59,84,37,8.05,scattered clouds
12,12,Cayenne,GF,4.9333,-52.3333,81.09,56,75,3.00,broken clouds
18,18,Avarua,CK,-21.2078,-159.7750,82.45,78,75,6.91,broken clouds
19,19,Kapaa,US,22.0752,-159.3190,82.11,71,20,9.22,few clouds
22,22,Cap Malheureux,MU,-19.9842,57.6142,77.47,88,3,1.01,clear sky
23,23,Pakokku,MM,21.3333,95.1000,77.29,34,7,5.55,clear sky
25,25,Sokoto,NG,13.0609,5.2390,80.51,17,54,10.25,broken clouds
26,26,Sibolga,ID,1.7427,98.7792,76.64,87,100,1.57,overcast clouds
29,29,Atuona,PF,-9.8000,-139.0333,78.31,73,16,15.12,few clouds


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                186
City                   186
Country                184
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Current Description    186
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

nan_value = float("NaN")
clean_df = preferred_cities_df.copy()

clean_df.replace("", nan_value, inplace=True)
clean_df.dropna(inplace=True)
clean_df.count()

City_ID                184
City                   184
Country                184
Lat                    184
Lng                    184
Max Temp               184
Humidity               184
Cloudiness             184
Wind Speed             184
Current Description    184
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,78.75,few clouds,-23.1203,-134.9692,
3,Saint-Philippe,RE,76.59,scattered clouds,-21.3585,55.7679,
12,Cayenne,GF,81.09,broken clouds,4.9333,-52.3333,
18,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,
19,Kapaa,US,82.11,few clouds,22.0752,-159.3190,
22,Cap Malheureux,MU,77.47,clear sky,-19.9842,57.6142,
23,Pakokku,MM,77.29,clear sky,21.3333,95.1000,
25,Sokoto,NG,80.51,broken clouds,13.0609,5.2390,
26,Sibolga,ID,76.64,overcast clouds,1.7427,98.7792,
29,Atuona,PF,78.31,few clouds,-9.8000,-139.0333,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
hotel_df.count()

City                   184
Country                184
Max Temp               184
Current Description    184
Lat                    184
Lng                    184
Hotel Name             184
dtype: int64

In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()

clean_hotel_df.replace("", nan_value, inplace=True)
clean_hotel_df.dropna(inplace=True)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,78.75,few clouds,-23.1203,-134.9692,People ThankYou
3,Saint-Philippe,RE,76.59,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
12,Cayenne,GF,81.09,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
18,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,Paradise Inn
19,Kapaa,US,82.11,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
667,Ahuimanu,US,85.14,scattered clouds,21.4447,-157.8378,Paradise Bay Resort
671,Bandar-E Lengeh,IR,75.31,scattered clouds,26.5579,54.8807,کافی نت پارس
674,Kombissiri,BF,82.29,scattered clouds,12.0656,-1.3375,Mitibkieta Annexe
679,Batticaloa,LK,78.49,overcast clouds,7.7102,81.6924,Grand Star Guest House


In [78]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Data_CSV/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [76]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [77]:
# 11a. Add a marker layer for each city to the map. 
hotels = clean_hotel_df["Hotel Name"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))